# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770356819249                   -0.53    9.0    171ms
  2   -2.771681790027       -2.88       -1.31    1.0    118ms
  3   -2.771714132553       -4.49       -2.61    1.0    104ms
  4   -2.771714687478       -6.26       -4.14    1.0    102ms
  5   -2.771714715193       -7.56       -4.61    2.0    123ms
  6   -2.771714715246      -10.28       -5.61    1.0    181ms
  7   -2.771714715249      -11.43       -5.96    2.0    858ms
  8   -2.771714715250      -12.77       -6.77    1.0    104ms
  9   -2.771714715250   +  -13.85       -7.41    1.0    127ms
 10   -2.771714715250   +  -15.05       -7.92    2.0    154ms
 11   -2.771714715250      -14.45       -8.86    1.0    141ms


-0.00013457363766708575

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770509885643                   -0.53    9.0    944ms
  2   -2.771775169119       -2.90       -1.30    1.0    194ms
  3   -2.771801461985       -4.58       -2.66    1.0    100ms
  4   -2.771802043066       -6.24       -4.18    1.0    124ms
  5   -2.771802074445       -7.50       -5.08    2.0    117ms
  6   -2.771802074476      -10.51       -6.01    2.0    132ms
  7   -2.771802074476      -12.89       -6.83    1.0    107ms
  8   -2.771802074476      -13.78       -7.62    2.0    122ms
  9   -2.771802074476   +  -14.01       -8.08    1.0    110ms


0.017612220210287556

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457363766708575
Displaced dipole:  0.017612220210287556
Polarizability :   1.774679384795464


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  61.6ms  Non-interacting
   1        0       1       -0.60     10.0   1.15s  
   2        0       2       -2.42      7.0   125ms  
   3        0       3       -3.55      5.0   115ms  
   4        0       4       -5.33      4.0   102ms  
   5        0       5       -7.94      1.0  85.6ms  
   6        0       6      -10.19      1.0  89.2ms  
   7        1       1       -7.45     13.0   221ms  Restart
   8        1       2       -8.92      1.0  87.3ms  
                                      13.0   123ms  Final orbitals


(δψ = Matrix{ComplexF64}[[0.0007902165220081817 + 0.0031997627814529025im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.30098454695625093 - 0.07165540773965466im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; 0.03990676222834685 - 0.009684894032728144im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.0785025443355514 + 0.019706405096473978im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.6206949544616146e-7 -2.502015469684257e-7 … -4.8806442728139593e-8 -2.502015544805671e-7; -6.293617526519254e-7 -4.793036923028625e-7 … -1.2077844714892534e-7 -4.793036971183115e-7; … ; 1.3626261050629337e-7 1.135622961256598e-7 … 4.711989230104602e-8 1.1356229348427168e-7; 1.330547524447203e-7 1.3632639647572015e-7 … 5.200684787299534e-8 1.3632638996150758e-7;;; -2.502015419794782e-7 -1.7404909749895445e-7 … -3.556494948007348e-8 -1.7404910281287497e-7; -4.793036891682948e-7 -3.6797322866297114e-7 … -1.1202895112723201e-7 -3.6797323193595473e-7; … ; 1.1356229591749928e-7 1.1211097800159403e-7 … 1.0980682122403172e-7 1.12

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125265939101
Interacting polarizability:     1.7736548573989688


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 3.54e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 5.31e-11
[ Info: GMRES linsolve in iteration 2; step 2: normres = 5.01e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ * norm of residual = 1.88e-13
└ * number of operations = 12
Non-interacting polarizability: 1.925712526593875
Interacting polarizability:     1.773654859335691


We obtain the identical result to above.